## POBLANDO CAPA FUNCTIONAL

**PASO 15°** Obteniendo información requeridad para el analisis de **Salario x Empresa**

**15.1** Definimos ruta tablas requeridas apuntando a la capa curated 

In [99]:
ruta_curated_empresas_google_cloud = "gs://introduccion-apache-spark/datalake/curated/empresas/"

ruta_curated_personas_google_cloud = "gs://introduccion-apache-spark/datalake/curated/personas/"

**15.2** Creamos el dataframe para cada tabla.

In [100]:
df_personas_curated = spark.read.format("parquet").option("header","true").load(ruta_curated_personas_google_cloud)

df_empresas_curated = spark.read.format("parquet").option("header","true").load(ruta_curated_empresas_google_cloud)

**15.3** Mostramos datos de los dataframes

In [101]:
df_personas_curated.show(5)
df_empresas_curated.show(5)

+---+---------+-----------+--------------------+-------------+----+-------+----------+
| ID|   NOMBRE|   telefono|              CORREO|FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+---------+-----------+--------------------+-------------+----+-------+----------+
|  1|     Carl|17456339145|arcu.Sed.et@ante....|   2004-04-23|  32|20095.0|         5|
|  2|Priscilla|    1552498|Donec.egestas.Ali...|   2019-02-17|  34| 9298.0|         2|
|  3|  Jocelyn|12049568594|amet.diam@loborti...|   2002-08-01|  27|10853.0|         3|
|  4|    Aidan|17198629385|euismod.et.commod...|   2018-11-06|  29| 3387.0|        10|
|  5|  Leandra|    8398044|at@pretiumetrutru...|   2002-10-10|  41|22102.0|         1|
+---+---------+-----------+--------------------+-------------+----+-------+----------+
only showing top 5 rows

+---+------------+
| ID|EMPRESA_NAME|
+---+------------+
|  5|      AMAZON|
|  6|      GOOGLE|
|  4|      TOYOTA|
|  7|     SAMSUNG|
|  3|       APPLE|
+---+------------+
only showing top 5 ro

**15.4** Realizar la unión (join) de ambas tablas 

In [103]:
df_join = df_personas_curated.join(df_empresas_curated, df_personas_curated.ID_EMPRESA == df_empresas_curated.ID)
df_join.show()

+---+---------+-----------+--------------------+-------------+----+-------+----------+---+------------+
| ID|   NOMBRE|   telefono|              CORREO|FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA| ID|EMPRESA_NAME|
+---+---------+-----------+--------------------+-------------+----+-------+----------+---+------------+
|  1|     Carl|17456339145|arcu.Sed.et@ante....|   2004-04-23|  32|20095.0|         5|  5|      AMAZON|
|  2|Priscilla|    1552498|Donec.egestas.Ali...|   2019-02-17|  34| 9298.0|         2|  2|   MICROSOFT|
|  3|  Jocelyn|12049568594|amet.diam@loborti...|   2002-08-01|  27|10853.0|         3|  3|       APPLE|
|  4|    Aidan|17198629385|euismod.et.commod...|   2018-11-06|  29| 3387.0|        10| 10|        SONY|
|  5|  Leandra|    8398044|at@pretiumetrutru...|   2002-10-10|  41|22102.0|         1|  1|     WALMART|
|  6|     Bert|    7974453|a.felis.ullamcorp...|   2017-04-25|  70| 7800.0|         7|  7|     SAMSUNG|
|  7|     Mark|16801026792|Quisque.ac@placer...|   2006-04-21|  

**15.5**  Realizar la unión (join) de ambas tablas utilizando **Spark SQL**

In [104]:
df_personas_curated.createOrReplaceTempView("tb_personas")
df_empresas_curated.createOrReplaceTempView("tb_empresas")

df_sql = spark.sql("SELECT * FROM tb_personas p inner join tb_empresas e on e.ID = p.ID_EMPRESA")
df_sql.show(5)

+---+---------+-----------+--------------------+-------------+----+-------+----------+---+------------+
| ID|   NOMBRE|   telefono|              CORREO|FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA| ID|EMPRESA_NAME|
+---+---------+-----------+--------------------+-------------+----+-------+----------+---+------------+
|  1|     Carl|17456339145|arcu.Sed.et@ante....|   2004-04-23|  32|20095.0|         5|  5|      AMAZON|
|  2|Priscilla|    1552498|Donec.egestas.Ali...|   2019-02-17|  34| 9298.0|         2|  2|   MICROSOFT|
|  3|  Jocelyn|12049568594|amet.diam@loborti...|   2002-08-01|  27|10853.0|         3|  3|       APPLE|
|  4|    Aidan|17198629385|euismod.et.commod...|   2018-11-06|  29| 3387.0|        10| 10|        SONY|
|  5|  Leandra|    8398044|at@pretiumetrutru...|   2002-10-10|  41|22102.0|         1|  1|     WALMART|
+---+---------+-----------+--------------------+-------------+----+-------+----------+---+------------+
only showing top 5 rows



**15.6** Seleccionar campos requeridos para el análisis 

In [105]:
df_select = df_join.select(col('EDAD'),col('SALARIO'),col('EMPRESA_NAME'))
df_select.show()

+----+-------+------------+
|EDAD|SALARIO|EMPRESA_NAME|
+----+-------+------------+
|  32|20095.0|      AMAZON|
|  34| 9298.0|   MICROSOFT|
|  27|10853.0|       APPLE|
|  29| 3387.0|        SONY|
|  41|22102.0|     WALMART|
|  70| 7800.0|     SAMSUNG|
|  52| 8112.0|      AMAZON|
|  23|17040.0|      AMAZON|
|  69| 6834.0|       APPLE|
|  19| 7996.0|     SAMSUNG|
|  48| 4913.0|          HP|
|  24|19943.0|          HP|
|  34| 9501.0|      AMAZON|
|  59|16289.0|   MICROSOFT|
|  27| 1539.0|      AMAZON|
|  26| 3377.0|   MICROSOFT|
|  60| 6851.0|      GOOGLE|
|  34| 4759.0|     SAMSUNG|
|  70|17403.0|      TOYOTA|
|  24|18752.0|         IBM|
+----+-------+------------+
only showing top 20 rows



**15.7** Definimos la ruta de functional y persistimos los registros

In [106]:
ruta_functional = "gs://introduccion-apache-spark/datalake/functional/salario_empresa/"

df_select.repartition(1).write.mode("overwrite").format("parquet").save(ruta_functional)

**15.8** Consultamos la información almacenada en Funcional

In [107]:
df_salario_empresa = spark.read.format("parquet").option("header","true").load(ruta_functional)
df_salario_empresa.show(10)

+----+-------+------------+
|EDAD|SALARIO|EMPRESA_NAME|
+----+-------+------------+
|  32|20095.0|      AMAZON|
|  34| 9298.0|   MICROSOFT|
|  27|10853.0|       APPLE|
|  29| 3387.0|        SONY|
|  41|22102.0|     WALMART|
|  70| 7800.0|     SAMSUNG|
|  52| 8112.0|      AMAZON|
|  23|17040.0|      AMAZON|
|  69| 6834.0|       APPLE|
|  19| 7996.0|     SAMSUNG|
+----+-------+------------+
only showing top 10 rows




# Felicitaciones por completar el curso de Introducción Apache Spark
#### **Elaborado por** Juan Salinas
#### Linkedin: https://www.linkedin.com/in/juan-salinas/